IMPLEMENTATION OF SIMUATLED ANNEALING ALGORITHM FOR 8 QUEENS PROBLEM

In [5]:
import random
import math
import matplotlib.pyplot as plt
import numpy as np
import time

In [5]:
html_code = """ 

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>3D Chessboard with 8 Queens - Simulated Annealing</title>
    <style>
      body {
        margin: 0;
        overflow: hidden;
        background: radial-gradient(ellipse at center, #0f0f1e 0%, #050510 100%);
        font-family: 'Georgia', serif;
      }
      canvas {
        display: block;
      }
      #info {
        position: absolute;
        top: 20px;
        left: 50%;
        transform: translateX(-50%);
        color: #ffd700;
        font-size: 24px;
        text-shadow: 0 0 10px rgba(255, 215, 0, 0.5);
        z-index: 100;
        pointer-events: none;
        letter-spacing: 2px;
      }
      #controls {
        position: absolute;
        bottom: 20px;
        right: 20px;
        color: #ffd700;
        z-index: 100;
        background: rgba(10, 10, 21, 0.8);
        padding: 12px;
        border-radius: 8px;
        border: 2px solid #ffd700;
        min-width: 180px;
      }
      #controls button {
        background: #ffd700;
        color: #0a0a15;
        border: none;
        padding: 6px 12px;
        margin: 3px 0;
        cursor: pointer;
        font-family: 'Georgia', serif;
        font-size: 12px;
        border-radius: 4px;
        width: 100%;
        font-weight: bold;
      }
      #controls button:hover {
        background: #ffed4e;
      }
      #controls button:disabled {
        background: #666;
        cursor: not-allowed;
      }
      .stat {
        margin: 6px 0;
        font-size: 11px;
      }
      .label {
        color: #c0c0c0;
      }
      .value {
        color: #ffd700;
        font-weight: bold;
      }
    </style>
  </head>
  <body>
    <div id="info">♛ 8 QUEENS - SIMULATED ANNEALING ♛</div>
    <div id="controls">
      <button id="solveBtn">Solve with Simulated Annealing</button>
      <button id="stepBtn">Step Forward</button>
      <button id="resetBtn">Reset</button>
      <div class="stat">
        <span class="label">Conflicts:</span>
        <span class="value" id="conflicts">-</span>
      </div>
      <div class="stat">
        <span class="label">Temperature:</span>
        <span class="value" id="temperature">-</span>
      </div>
      <div class="stat">
        <span class="label">Step:</span>
        <span class="value" id="step">-</span>
      </div>
      <div class="stat">
        <span class="label">Status:</span>
        <span class="value" id="status">Ready</span>
      </div>
    </div>
    <script type="module">
      import * as THREE from "https://cdn.jsdelivr.net/npm/three@0.164.1/build/three.module.js";

      const scene = new THREE.Scene();
      scene.background = new THREE.Color(0x0a0a15);
      scene.fog = new THREE.FogExp2(0x0a0a15, 0.02);
      const camera = new THREE.PerspectiveCamera(45, window.innerWidth / window.innerHeight, 0.1, 1000);
      const renderer = new THREE.WebGLRenderer({ antialias: true });
      renderer.setSize(window.innerWidth, window.innerHeight);
      renderer.setPixelRatio(window.devicePixelRatio);
      renderer.shadowMap.enabled = true;
      renderer.shadowMap.type = THREE.PCFSoftShadowMap;
      document.body.appendChild(renderer.domElement);

      const ambientLight = new THREE.AmbientLight(0xffffff, 0.6);
      scene.add(ambientLight);
      
      const mainLight = new THREE.DirectionalLight(0xffffff, 1.2);
      mainLight.position.set(5, 15, 5);
      mainLight.castShadow = true;
      mainLight.shadow.mapSize.width = 4096;
      mainLight.shadow.mapSize.height = 4096;
      scene.add(mainLight);
      
      const fillLight = new THREE.DirectionalLight(0xffffff, 0.4);
      fillLight.position.set(-5, 10, -5);
      scene.add(fillLight);

      const boardSize = 8;
      const tileSize = 1;
      const board = new THREE.Group();

      const whiteMat = new THREE.MeshStandardMaterial({ 
        color: 0xf5deb3,
        roughness: 0.2,
        metalness: 0.3,
        emissive: 0x332211,
        emissiveIntensity: 0.05
      });
      const blackMat = new THREE.MeshStandardMaterial({ 
        color: 0x3d2817,
        roughness: 0.3,
        metalness: 0.4,
        emissive: 0x110805,
        emissiveIntensity: 0.1
      });

      const borderGeometry = new THREE.BoxGeometry(boardSize + 0.6, 0.3, boardSize + 0.6);
      const borderMaterial = new THREE.MeshStandardMaterial({ 
        color: 0x2c1810,
        roughness: 0.6,
        metalness: 0.2
      });
      const border = new THREE.Mesh(borderGeometry, borderMaterial);
      border.position.y = -0.1;
      border.castShadow = true;
      border.receiveShadow = true;
      board.add(border);
      
      const inlayGeometry = new THREE.BoxGeometry(boardSize + 0.5, 0.32, boardSize + 0.5);
      const inlayMaterial = new THREE.MeshStandardMaterial({ 
        color: 0xffd700,
        roughness: 0.3,
        metalness: 0.8,
        emissive: 0xffd700,
        emissiveIntensity: 0.1
      });
      const inlay = new THREE.Mesh(inlayGeometry, inlayMaterial);
      inlay.position.y = -0.09;
      board.add(inlay);

      for (let i = 0; i < boardSize; i++) {
        for (let j = 0; j < boardSize; j++) {
          const geometry = new THREE.BoxGeometry(tileSize, 0.15, tileSize);
          const material = (i + j) % 2 === 0 ? whiteMat : blackMat;
          const tile = new THREE.Mesh(geometry, material);
          tile.position.set(i - boardSize / 2 + 0.5, 0.025, j - boardSize / 2 + 0.5);
          tile.receiveShadow = true;
          tile.castShadow = true;
          board.add(tile);
        }
      }
      scene.add(board);

      const createQueen = () => {
        const queenGroup = new THREE.Group();
        
        const queenMaterial = new THREE.MeshStandardMaterial({ 
          color: 0xffd700,
          metalness: 0.95,
          roughness: 0.15,
          envMapIntensity: 1.0
        });
        
        const accentMaterial = new THREE.MeshStandardMaterial({
          color: 0xc0c0c0,
          metalness: 0.9,
          roughness: 0.2
        });

        const baseGroup = new THREE.Group();
        
        const platformGeometry = new THREE.CylinderGeometry(0.38, 0.42, 0.08, 64);
        const platform = new THREE.Mesh(platformGeometry, accentMaterial);
        platform.position.y = 0.04;
        platform.castShadow = true;
        baseGroup.add(platform);

        const baseGeometry = new THREE.CylinderGeometry(0.32, 0.38, 0.18, 64);
        const base = new THREE.Mesh(baseGeometry, queenMaterial);
        base.position.y = 0.17;
        base.castShadow = true;
        baseGroup.add(base);

        const baseRingGeometry = new THREE.TorusGeometry(0.33, 0.025, 16, 64);
        const baseRing = new THREE.Mesh(baseRingGeometry, accentMaterial);
        baseRing.position.y = 0.26;
        baseRing.rotation.x = Math.PI / 2;
        baseRing.castShadow = true;
        baseGroup.add(baseRing);

        const lowerNeckGeometry = new THREE.CylinderGeometry(0.18, 0.28, 0.25, 64);
        const lowerNeck = new THREE.Mesh(lowerNeckGeometry, queenMaterial);
        lowerNeck.position.y = 0.425;
        lowerNeck.castShadow = true;
        baseGroup.add(lowerNeck);

        const midGeometry = new THREE.CylinderGeometry(0.14, 0.18, 0.3, 64);
        const mid = new THREE.Mesh(midGeometry, queenMaterial);
        mid.position.y = 0.675;
        mid.castShadow = true;
        baseGroup.add(mid);

        [0.55, 0.8].forEach(yPos => {
          const bandGeometry = new THREE.TorusGeometry(0.15, 0.02, 16, 64);
          const band = new THREE.Mesh(bandGeometry, accentMaterial);
          band.position.y = yPos;
          band.rotation.x = Math.PI / 2;
          band.castShadow = true;
          baseGroup.add(band);
        });

        queenGroup.add(baseGroup);

        const headGroup = new THREE.Group();
        headGroup.position.y = 0.82;
        
        const upperNeckGeometry = new THREE.CylinderGeometry(0.12, 0.14, 0.2, 64);
        const upperNeck = new THREE.Mesh(upperNeckGeometry, queenMaterial);
        upperNeck.position.y = 0.1;
        upperNeck.castShadow = true;
        headGroup.add(upperNeck);

        const crownBaseGeometry = new THREE.CylinderGeometry(0.18, 0.2, 0.12, 32);
        const crownBase = new THREE.Mesh(crownBaseGeometry, queenMaterial);
        crownBase.position.y = 0.26;
        crownBase.castShadow = true;
        headGroup.add(crownBase);

        const crownBandGeometry = new THREE.TorusGeometry(0.19, 0.02, 16, 32);
        const crownBand = new THREE.Mesh(crownBandGeometry, accentMaterial);
        crownBand.position.y = 0.32;
        crownBand.rotation.x = Math.PI / 2;
        crownBand.castShadow = true;
        headGroup.add(crownBand);

        const pointCount = 5;
        for (let i = 0; i < pointCount; i++) {
          const angle = (i / pointCount) * Math.PI * 2;
          const radius = 0.17;
          
          const isCenterPoint = i === 0;
          const height = isCenterPoint ? 0.22 : 0.18;
          const baseWidth = 0.06;
          
          const pointGeometry = new THREE.ConeGeometry(baseWidth, height, 4);
          const crownPoint = new THREE.Mesh(pointGeometry, queenMaterial);
          
          const x = Math.cos(angle) * radius;
          const z = Math.sin(angle) * radius;
          crownPoint.position.set(x, 0.32 + height / 2, z);
          
          crownPoint.castShadow = true;
          headGroup.add(crownPoint);
          
          const tipGeometry = new THREE.SphereGeometry(0.03, 16, 16);
          const tip = new THREE.Mesh(tipGeometry, accentMaterial);
          tip.position.set(x, 0.32 + height, z);
          tip.castShadow = true;
          headGroup.add(tip);
        }

        queenGroup.add(headGroup);
        queenGroup.userData.headGroup = headGroup;

        return queenGroup;
      };

      const queens = [];
      for (let i = 0; i < 8; i++) {
        const queen = createQueen();
        queen.traverse((child) => {
          if (child.isMesh) {
            child.castShadow = true;
          }
        });
        board.add(queen);
        queens.push(queen);
      }

      const addBackgroundEffects = () => {
        const floorGeometry = new THREE.CircleGeometry(25, 64);
        const floorMaterial = new THREE.MeshStandardMaterial({
          color: 0x0a0a15,
          roughness: 0.1,
          metalness: 0.9,
          side: THREE.DoubleSide
        });
        const floor = new THREE.Mesh(floorGeometry, floorMaterial);
        floor.rotation.x = -Math.PI / 2;
        floor.position.y = -0.5;
        floor.receiveShadow = true;
        scene.add(floor);
        
        const particleCount = 200;
        const particles = new THREE.BufferGeometry();
        const positions = new Float32Array(particleCount * 3);
        const colors = new Float32Array(particleCount * 3);
        
        for (let i = 0; i < particleCount; i++) {
          positions[i * 3] = (Math.random() - 0.5) * 50;
          positions[i * 3 + 1] = Math.random() * 30;
          positions[i * 3 + 2] = (Math.random() - 0.5) * 50;
          
          const colorChoice = Math.random();
          if (colorChoice > 0.8) {
            colors[i * 3] = 0.25;
            colors[i * 3 + 1] = 0.41;
            colors[i * 3 + 2] = 0.88;
          } else if (colorChoice > 0.6) {
            colors[i * 3] = 0.54;
            colors[i * 3 + 1] = 0;
            colors[i * 3 + 2] = 0.54;
          } else {
            colors[i * 3] = 1;
            colors[i * 3 + 1] = 0.84;
            colors[i * 3 + 2] = 0;
          }
        }
        
        particles.setAttribute('position', new THREE.BufferAttribute(positions, 3));
        particles.setAttribute('color', new THREE.BufferAttribute(colors, 3));
        
        const particleMaterial = new THREE.PointsMaterial({
          size: 0.15,
          transparent: true,
          opacity: 0.6,
          vertexColors: true,
          blending: THREE.AdditiveBlending,
          sizeAttenuation: true
        });
        
        window.particleSystem = new THREE.Points(particles, particleMaterial);
        scene.add(window.particleSystem);
      };

      addBackgroundEffects();

      camera.position.set(8, 10, 8);
      camera.lookAt(0, 0, 0);

      let mouseX = 0, mouseY = 0;
      let targetRotationX = 0, targetRotationY = 0;
      const windowHalfX = window.innerWidth / 2;
      const windowHalfY = window.innerHeight / 2;

      document.addEventListener('mousemove', (event) => {
        mouseX = (event.clientX - windowHalfX) / 100;
        mouseY = (event.clientY - windowHalfY) / 100;
      });

      let currentState = [];
      let currentTemp = 100;
      let currentStep = 0;
      let isRunning = false;
      let animationSpeed = 100;

      function conflicts(state) {
        let conflict = 0;
        const n = state.length;
        for (let i = 0; i < n; i++) {
          for (let j = i + 1; j < n; j++) {
            if (state[i] === state[j] || Math.abs(state[i] - state[j]) === Math.abs(i - j)) {
              conflict++;
            }
          }
        }
        return conflict;
      }

      function randomState() {
        return Array.from({length: 8}, () => Math.floor(Math.random() * 8));
      }

      function randomNeighbor(state) {
        const newState = [...state];
        const row = Math.floor(Math.random() * 8);
        let newCol = Math.floor(Math.random() * 8);
        while (newCol === state[row]) {
          newCol = Math.floor(Math.random() * 8);
        }
        newState[row] = newCol;
        return newState;
      }

      function updateQueenPositions(state) {
        for (let row = 0; row < 8; row++) {
          const col = state[row];
          queens[row].position.x = col - boardSize / 2 + 0.5;
          queens[row].position.z = row - boardSize / 2 + 0.5;
        }
      }

      function updateUI() {
        document.getElementById('conflicts').textContent = conflicts(currentState);
        document.getElementById('temperature').textContent = currentTemp.toFixed(2);
        document.getElementById('step').textContent = currentStep;
      }

      function resetAlgorithm() {
        currentState = randomState();
        currentTemp = 100;
        currentStep = 0;
        isRunning = false;
        updateQueenPositions(currentState);
        updateUI();
        document.getElementById('status').textContent = 'Ready';
        document.getElementById('solveBtn').disabled = false;
        document.getElementById('stepBtn').disabled = false;
      }

      function performStep() {
        if (conflicts(currentState) === 0) {
          document.getElementById('status').textContent = '✅ Solution Found!';
          isRunning = false;
          document.getElementById('solveBtn').disabled = false;
          document.getElementById('stepBtn').disabled = false;
          return false;
        }

        const neighbor = randomNeighbor(currentState);
        const currentConflicts = conflicts(currentState);
        const neighborConflicts = conflicts(neighbor);
        const delta = neighborConflicts - currentConflicts;

        if (delta < 0) {
          currentState = neighbor;
        } else {
          const prob = Math.exp(-delta / currentTemp);
          if (Math.random() < prob) {
            currentState = neighbor;
          }
        }

        currentTemp *= 0.95;
        currentStep++;

        updateQueenPositions(currentState);
        updateUI();

        if (currentTemp < 0.0001) {
          document.getElementById('status').textContent = '❌ Stopped (temp too low)';
          isRunning = false;
          document.getElementById('solveBtn').disabled = false;
          document.getElementById('stepBtn').disabled = false;
          return false;
        }

        return true;
      }

      function solveWithAnimation() {
        if (!isRunning) return;
        
        const canContinue = performStep();
        
        if (canContinue && isRunning) {
          setTimeout(solveWithAnimation, animationSpeed);
        }
      }

      document.getElementById('solveBtn').addEventListener('click', () => {
        if (currentStep === 0) {
          resetAlgorithm();
        }
        isRunning = true;
        document.getElementById('status').textContent = '⚙️ Solving...';
        document.getElementById('solveBtn').disabled = true;
        document.getElementById('stepBtn').disabled = true;
        solveWithAnimation();
      });

      document.getElementById('stepBtn').addEventListener('click', () => {
        if (currentStep === 0) {
          resetAlgorithm();
        }
        performStep();
      });

      document.getElementById('resetBtn').addEventListener('click', () => {
        isRunning = false;
        resetAlgorithm();
      });

      resetAlgorithm();

      function animate() {
        requestAnimationFrame(animate);
        
        const time = Date.now() * 0.001;
        
        targetRotationY = mouseX * 0.0003;
        targetRotationX = mouseY * 0.0003;
        
        board.rotation.y += targetRotationY;
        board.rotation.x += targetRotationX;
        
        board.rotation.y *= 0.95;
        board.rotation.x *= 0.95;
        
        board.rotation.y += 0.002;
        
        queens.forEach((queen, index) => {
          const headGroup = queen.userData.headGroup;
          if (headGroup) {
            headGroup.rotation.y = Math.sin(time * 0.5 + index) * 0.1;
          }
        });
        
        if (window.particleSystem) {
          const positions = window.particleSystem.geometry.attributes.position.array;
          for (let i = 0; i < positions.length; i += 3) {
            positions[i + 1] += Math.sin(time + i) * 0.01;
            if (positions[i + 1] > 30) {
              positions[i + 1] = 0;
            }
          }
          window.particleSystem.geometry.attributes.position.needsUpdate = true;
          window.particleSystem.rotation.y = time * 0.05;
        }

        renderer.render(scene, camera);
      }
      animate();

      window.addEventListener("resize", () => {
        camera.aspect = window.innerWidth / window.innerHeight;
        camera.updateProjectionMatrix();
        renderer.setSize(window.innerWidth, window.innerHeight);
      });
    </script>
  </body>
</html>
"""

with open("eight_queens_3d.html", "w", encoding="utf-8") as f:
    f.write(html_code)
print("✅ File created: eight_queens_3d.html")

✅ File created: eight_queens_3d.html


In [6]:
import webbrowser
webbrowser.open("eight_queens_3d.html")

True